# A simple bottleneck

Very simple code that :
- make_random_edges : generate a random graph given a number of edge and nodes
- find_unique_edges : filter the list of edges to only keep an instance of each (i.e. remove duplicate edges)
- contains : verify if an edge is part of a list

In [15]:
import random

def make_random_edges(n_edges=100, n_nodes=10):
    random.seed(42)
    edges = [[random.randint(0, n_nodes), random.randint(0, n_nodes)] for e in range(n_edges)]
    return edges

def find_unique_edges(edges):
    edges = list(edges)
    unique_edges = []
    while edges:
        edge = edges.pop()
        if not contains(edges, edge):
            unique_edges.append(edge)
    return unique_edges

def contains(edges, edge):
    for e in edges:
        if sorted(e) == sorted(edge):
            return True
    return False


Some profiling functions to ease the process

In [16]:
from functools import wraps
from cProfile import Profile
from tempfile import NamedTemporaryFile
import pstats

_time_profiles = {}

def profile_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        profile = Profile()
        ret = profile.runcall(func, *args, **kwargs)
        _time_profiles[(wrapper, ) + args] = profile
        
        return ret 
    return wrapper

def profile_stats(profile):
    temp_stats = NamedTemporaryFile()
    profile.dump_stats(temp_stats.name)
    return pstats.Stats(temp_stats.name)

Simple profiling with text outputs  

In [17]:
@profile_time
def remove_duplicate_edges(n_edges=2000):
    edges = make_random_edges(n_edges, 200)
    unique_edges = find_unique_edges(edges)
    return unique_edges

unique_edges = remove_duplicate_edges(2000)

profile = _time_profiles[(remove_duplicate_edges, 2000)]

stats = profile_stats(profile)
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()

Mon Dec 15 10:37:20 2025    /var/folders/nw/_bz483qd50b_9vt_bcqkwkx00000gn/T/tmp7zo57lna

         3930641 function calls in 0.375 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  3903622    0.209    0.000    0.209    0.000 {built-in method builtins.sorted}
     2000    0.161    0.000    0.369    0.000 2053488929.py:17(contains)
     4000    0.002    0.000    0.002    0.000 random.py:237(_randbelow_with_getrandbits)
     4000    0.002    0.000    0.004    0.000 random.py:290(randrange)
        1    0.001    0.001    0.005    0.005 2053488929.py:5(<listcomp>)
     4000    0.001    0.000    0.005    0.000 random.py:334(randint)
        1    0.001    0.001    0.370    0.370 2053488929.py:8(find_unique_edges)
     5051    0.000    0.000    0.000    0.000 {method 'getrandbits' of '_random.Random' objects}
     4000    0.000    0.000    0.000    0.000 {method 'bit_length' of 'int' objects}
     1919    0.000    0.000    0.000   

Advance profiling with visualization

In [18]:
%reload_ext snakeviz

In [19]:
from snakeviz.ipymagic import open_snakeviz_and_display_in_notebook

def display_stats(profile):
    temp_stats = NamedTemporaryFile(delete=False)
    profile.dump_stats(temp_stats.name)
    return open_snakeviz_and_display_in_notebook(temp_stats.name)

In [20]:
@profile_time
def remove_duplicate_edges(n_edges=2000):
    edges = make_random_edges(n_edges, 200)
    unique_edges = find_unique_edges(edges)
    return unique_edges

unique_edges = remove_duplicate_edges(2000)

profile = _time_profiles[(remove_duplicate_edges, 2000)]
display_stats(profile)

<function display at 0x102657550>


<Popen: returncode: None args: ['/Users/selimbenbouzid/Work/5IF/Parallel-com...>

Propose a new implementation and profile it

In [24]:
def find_unique_edges_improved(edges):
    edges = sorted([sorted(e) for e in edges])
    unique_edges = []
    if (len(edges)):
        unique_edges = edges[0]
    for i in range(1, len(edges)):
        if edges[i] == unique_edges[-1]:
            continue
        unique_edges.append(edges[i])
    return unique_edges

In [25]:
@profile_time
def remove_duplicate_edges_improved(n_edges=2000):
    edges = make_random_edges(n_edges, 200)
    unique_edges = find_unique_edges_improved(edges)
    return unique_edges

unique_edges = remove_duplicate_edges_improved(2000)

profile = _time_profiles[(remove_duplicate_edges_improved, 2000)]

stats = profile_stats(profile)
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()

Mon Dec 15 10:52:09 2025    /var/folders/nw/_bz483qd50b_9vt_bcqkwkx00000gn/T/tmpjms8zpbc

         25082 function calls in 0.006 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     4000    0.002    0.000    0.003    0.000 random.py:290(randrange)
     4000    0.001    0.000    0.002    0.000 random.py:237(_randbelow_with_getrandbits)
     2001    0.001    0.000    0.001    0.000 {built-in method builtins.sorted}
     4000    0.001    0.000    0.004    0.000 random.py:334(randint)
        1    0.001    0.001    0.005    0.005 2053488929.py:5(<listcomp>)
        1    0.000    0.000    0.002    0.002 1429056555.py:1(find_unique_edges_improved)
     5051    0.000    0.000    0.000    0.000 {method 'getrandbits' of '_random.Random' objects}
        1    0.000    0.000    0.001    0.001 1429056555.py:2(<listcomp>)
     4000    0.000    0.000    0.000    0.000 {method 'bit_length' of 'int' objects}
     1918    0.000    0.000    

In [27]:
display(stats)